In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=0c4bf2e1b7574e4d178c483c9e006cf3c8ede507aa08a698ae57fe83a589ccaa
  Stored in directory: c:\users\abelo\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [102]:
!pip install xlsxwriter

In [1]:
import requests
import bs4

In [2]:
from tqdm import tqdm
import time
import xlsxwriter

In [3]:
sleep_time = 0.001

def collect_links(soup, exclude=set(), link_keyword='/wiki/'):
    links = []
    for link in soup.find_all('a'):
        l = link.get('href')
        title = link.get('title')
        cl = link.get('class')
        if l is None or link.text is None or title is None or cl is not None:
            continue
        if l.startswith(link_keyword) and ".png" not in l and ".jpg" not in l and l not in exclude and "Служебная" not in l:
            links.append("https://ru.wikipedia.org" + l)
        else:
            pass
    return links


def wiki_crawler(path, start_url, links_lim=1000):
    workbook = xlsxwriter.Workbook(path)
    worksheet = workbook.add_worksheet("Data")
    for col, item in enumerate(["prev_page_link", "current_page_link", "heading", "sub_headings", "images"]):
            worksheet.write(0, col, item)
    
    stack = [(start_url, None)]
    visited = 0
    visited_links = set()

    pbar = tqdm(total=links_lim)
    while len(stack) > 0 and visited <= links_lim:
        url, prev_url = stack.pop(0)
        visited_links.add(url) # not to visit again to prevent loops
        try:
            time.sleep(sleep_time)
            resp = requests.get(url)
        except e:
            print(f"Exception: {e}")
            continue
        
        # maknig a soup of the current website
        soup = bs4.BeautifulSoup(resp.content, 'html.parser')
        # check if page is valid
        heading = soup.find("span", {"class": "mw-page-title-main"})
        if heading is None:
            continue
            # print(f"Error with page: {url} ; \nNo heading!!!")
        else:
            heading = heading.text
            visited += 1
            pbar.update(1)
        
        # adding all the new links to the stack
        links = collect_links(soup, visited_links)
        for link in links:
            stack.append((link, url)) # new and prev urls
            
        # collecting the nesessary data
        sub_headings = soup.find_all("span", {"class": "mw-headline"})
        sub_headings = " ; ".join([sub_heading.text for sub_heading in sub_headings])
        
        images = soup.find_all("img")
        images = " ; ".join([str(image.get('src')).replace("https:", "") for image in images if image.get('src') is not None])
        
        for col, item in enumerate([prev_url, url, heading, sub_headings, images]):
            worksheet.write(visited, col, item)
        # print(len(stack))
    pbar.close()
    workbook.close()

In [4]:
start = "https://ru.wikipedia.org/wiki/%D0%A6%D0%B8%D0%B2%D0%B8%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F"
path = "./output1.xlsx"
wiki_crawler(path, start, links_lim=6000)

6001it [45:54,  2.18it/s]                            
